In [2]:
import torch
import os

In [3]:
import numpy as np

In [4]:
%load_ext autoreload
%autoreload 2

In [ ]:
/Users/yashsurange/Documents/GitHub/FER-from-static-images/CK+_Complete

In [3]:
sorted(os.listdir('/Users/yashsurange/Documents/GitHub/FER-from-static-images/CK+_Complete'))

['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']

In [24]:
os.listdir('/Users/yashsurange/Documents/GitHub/FER-from-static-images/')

['check.ipynb',
 'CK+_Complete',
 '__pycache__',
 'README.md',
 'dataset.py',
 '.gitignore',
 '.git']

In [5]:
from dataset import ckplus

In [7]:
test_data=ckplus('./CK+_Complete/')

In [8]:
np.array(test_data.__getitem__(0)[0]).reshape(480,720,1).shape

(480, 720, 1)

In [9]:
test_data.__len__()

1635

In [18]:
# width=720 and height=480
(test_data.__getitem__(0)[0]).size

(720, 480)

In [21]:
image=np.array((test_data.__getitem__(0)[0]))

In [36]:
np.all(np.stack((image,)*3,axis=-1)[:,:,2]==image)

True

In [39]:
np.stack((image,)*3,axis=0).shape

(3, 480, 720)

In [37]:
np.stack((image,)*3,axis=-1)[:,:,2].shape

(480, 720)

In [34]:
np.stack((image,)*3,axis=-1)[1].shape

(720, 3)

In [32]:
image

array([[ 72, 253, 254, ...,   2,   1,   0],
       [ 73, 253, 254, ...,   2,   1,   0],
       [ 73, 253, 254, ...,   2,   1,   0],
       ...,
       [ 72, 253, 254, ...,   1,   1,   1],
       [ 72, 253, 254, ...,   1,   1,   1],
       [ 72, 253, 254, ...,   1,   1,   1]], dtype=uint8)

In [43]:
test_image=np.array([[1,2,3],
                    [3,4,5]])
test_image.shape

(2, 3)

In [57]:
np.stack((test_image,)*2,axis=-1)

array([[[1, 1],
        [2, 2],
        [3, 3]],

       [[3, 3],
        [4, 4],
        [5, 5]]])